In [1]:
import xarray as xr
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob

# Read ATOMIC Saildrone data via openDaP  --- Not working ---

In [ ]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/insitu/L2/saildrone/Atomic//saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

# try using podaac drive (z:)  --- This works ----

In [10]:
url = 'Z:/allData/insitu/L2/saildrone/Atomic/saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

ds_usv

<xarray.Dataset>
Dimensions:                         (obs: 66240, trajectory: 1)
Coordinates:
    latitude                        (trajectory, obs) float64 ...
    longitude                       (trajectory, obs) float64 ...
    time                            (trajectory, obs) datetime64[ns] ...
  * trajectory                      (trajectory) float32 1061.0
Dimensions without coordinates: obs
Data variables:
    SOG                             (trajectory, obs) float64 ...
    SOG_FILTERED_MEAN               (trajectory, obs) float64 ...
    SOG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    SOG_FILTERED_MAX                (trajectory, obs) float64 ...
    SOG_FILTERED_MIN                (trajectory, obs) float64 ...
    COG                             (trajectory, obs) float64 ...
    COG_FILTERED_MEAN               (trajectory, obs) float64 ...
    COG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    HDG                             (trajectory, obs) float64 ...
    HDG_FILTERED_MEAN               (trajectory, obs) float64 ...
    HDG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    ROLL_FILTERED_MEAN              (trajectory, obs) float64 ...
    ROLL_FILTERED_STDDEV            (trajectory, obs) float64 ...
    ROLL_FILTERED_PEAK              (trajectory, obs) float64 ...
    PITCH_FILTERED_MEAN             (trajectory, obs) float64 ...
    PITCH_FILTERED_STDDEV           (trajectory, obs) float64 ...
    PITCH_FILTERED_PEAK             (trajectory, obs) float64 ...
    HDG_WING                        (trajectory, obs) float64 ...
    WING_HDG_FILTERED_MEAN          (trajectory, obs) float64 ...
    WING_HDG_FILTERED_STDDEV        (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_MEAN         (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_STDDEV       (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_PEAK         (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_MEAN        (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_STDDEV      (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_PEAK        (trajectory, obs) float64 ...
    WING_ANGLE                      (trajectory, obs) float64 ...
    UWND_MEAN                       (trajectory, obs) float64 ...
    UWND_STDDEV                     (trajectory, obs) float64 ...
    VWND_MEAN                       (trajectory, obs) float64 ...
    VWND_STDDEV                     (trajectory, obs) float64 ...
    WWND_MEAN                       (trajectory, obs) float64 ...
    WWND_STDDEV                     (trajectory, obs) float64 ...
    GUST_WND_MEAN                   (trajectory, obs) float64 ...
    GUST_WND_STDDEV                 (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN    (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_STDDEV  (trajectory, obs) float64 ...
    TEMP_AIR_MEAN                   (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV                 (trajectory, obs) float64 ...
    RH_MEAN                         (trajectory, obs) float64 ...
    RH_STDDEV                       (trajectory, obs) float64 ...
    BARO_PRES_MEAN                  (trajectory, obs) float64 ...
    BARO_PRES_STDDEV                (trajectory, obs) float64 ...
    PAR_AIR_MEAN                    (trajectory, obs) float64 ...
    PAR_AIR_STDDEV                  (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (trajectory, obs) float64 ...
    WAVE_DOMINANT_PERIOD            (trajectory, obs) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (trajectory, obs) float64 ...
    TEMP_SBE37_MEAN                 (trajectory, obs) float64 ...
    TEMP_SBE37_STDDEV               (trajectory, obs) float64 ...
    SAL_SBE37_MEAN                  (trajectory, obs) float64 ...
    SAL_SBE37_STDDEV                (trajectory, obs) float64 ...
    COND_SBE37_MEAN                 (trajectory, obs) float64 ...
    COND_SBE37_STDDEV

# try using file downloaded locally (from opendap link) -- this works --

In [11]:
url = './../data/saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

ds_usv

<xarray.Dataset>
Dimensions:                         (obs: 66240, trajectory: 1)
Coordinates:
    latitude                        (trajectory, obs) float64 ...
    longitude                       (trajectory, obs) float64 ...
    time                            (trajectory, obs) datetime64[ns] ...
  * trajectory                      (trajectory) float32 1061.0
Dimensions without coordinates: obs
Data variables:
    SOG                             (trajectory, obs) float64 ...
    SOG_FILTERED_MEAN               (trajectory, obs) float64 ...
    SOG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    SOG_FILTERED_MAX                (trajectory, obs) float64 ...
    SOG_FILTERED_MIN                (trajectory, obs) float64 ...
    COG                             (trajectory, obs) float64 ...
    COG_FILTERED_MEAN               (trajectory, obs) float64 ...
    COG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    HDG                             (trajectory, obs) float64 ...
    HDG_FILTERED_MEAN               (trajectory, obs) float64 ...
    HDG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    ROLL_FILTERED_MEAN              (trajectory, obs) float64 ...
    ROLL_FILTERED_STDDEV            (trajectory, obs) float64 ...
    ROLL_FILTERED_PEAK              (trajectory, obs) float64 ...
    PITCH_FILTERED_MEAN             (trajectory, obs) float64 ...
    PITCH_FILTERED_STDDEV           (trajectory, obs) float64 ...
    PITCH_FILTERED_PEAK             (trajectory, obs) float64 ...
    HDG_WING                        (trajectory, obs) float64 ...
    WING_HDG_FILTERED_MEAN          (trajectory, obs) float64 ...
    WING_HDG_FILTERED_STDDEV        (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_MEAN         (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_STDDEV       (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_PEAK         (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_MEAN        (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_STDDEV      (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_PEAK        (trajectory, obs) float64 ...
    WING_ANGLE                      (trajectory, obs) float64 ...
    UWND_MEAN                       (trajectory, obs) float64 ...
    UWND_STDDEV                     (trajectory, obs) float64 ...
    VWND_MEAN                       (trajectory, obs) float64 ...
    VWND_STDDEV                     (trajectory, obs) float64 ...
    WWND_MEAN                       (trajectory, obs) float64 ...
    WWND_STDDEV                     (trajectory, obs) float64 ...
    GUST_WND_MEAN                   (trajectory, obs) float64 ...
    GUST_WND_STDDEV                 (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN    (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_STDDEV  (trajectory, obs) float64 ...
    TEMP_AIR_MEAN                   (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV                 (trajectory, obs) float64 ...
    RH_MEAN                         (trajectory, obs) float64 ...
    RH_STDDEV                       (trajectory, obs) float64 ...
    BARO_PRES_MEAN                  (trajectory, obs) float64 ...
    BARO_PRES_STDDEV                (trajectory, obs) float64 ...
    PAR_AIR_MEAN                    (trajectory, obs) float64 ...
    PAR_AIR_STDDEV                  (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (trajectory, obs) float64 ...
    WAVE_DOMINANT_PERIOD            (trajectory, obs) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (trajectory, obs) float64 ...
    TEMP_SBE37_MEAN                 (trajectory, obs) float64 ...
    TEMP_SBE37_STDDEV               (trajectory, obs) float64 ...
    SAL_SBE37_MEAN                  (trajectory, obs) float64 ...
    SAL_SBE37_STDDEV                (trajectory, obs) float64 ...
    COND_SBE37_MEAN                 (trajectory, obs) float64 ...
    COND_SBE37_STDDEV

# clean up usv file for interpolation
- calculate date range for files to read in

In [12]:
ds_usv = ds_usv.rename({'latitude':'lat','longitude':'lon'}).isel(trajectory=0).swap_dims({'obs':'time'})

ds_usv

2020-01-17T00:00:00.000000000 2020-03-02T23:59:00.000000000


<xarray.Dataset>
Dimensions:                         (time: 66240)
Coordinates:
    lat                             (time) float64 11.76 11.76 ... 9.192 9.193
    lon                             (time) float64 -56.09 -56.09 ... -52.02
  * time                            (time) datetime64[ns] 2020-01-17 ... 2020...
    trajectory                      float32 1061.0
Data variables:
    SOG                             (time) float64 2.108 1.967 ... 1.722 1.587
    SOG_FILTERED_MEAN               (time) float64 1.95 1.942 ... 1.749 1.678
    SOG_FILTERED_STDDEV             (time) float64 0.268 0.251 ... 0.32 0.299
    SOG_FILTERED_MAX                (time) float64 2.451 2.58 ... 2.64 2.539
    SOG_FILTERED_MIN                (time) float64 1.43 1.386 ... 1.177 1.027
    COG                             (time) float64 174.5 151.7 ... 314.8 318.1
    COG_FILTERED_MEAN               (time) float64 164.0 162.5 ... 316.0 318.8
    COG_FILTERED_STDDEV             (time) float64 12.6 12.2 15.0 ... 15.3 14.1
    HDG                             (time) float64 155.9 146.4 ... 321.8 311.9
    HDG_FILTERED_MEAN               (time) float64 155.8 147.0 ... 321.5 310.2
    HDG_FILTERED_STDDEV             (time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ROLL_FILTERED_MEAN              (time) float64 13.7 4.9 1.8 ... -15.0 2.9
    ROLL_FILTERED_STDDEV            (time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ROLL_FILTERED_PEAK              (time) float64 13.7 4.9 1.8 ... -15.0 2.9
    PITCH_FILTERED_MEAN             (time) float64 1.7 1.0 3.4 ... -1.3 -3.2
    PITCH_FILTERED_STDDEV           (time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PITCH_FILTERED_PEAK             (time) float64 1.7 1.0 3.4 ... -1.3 -3.2
    HDG_WING                        (time) float64 99.3 93.9 93.3 ... 61.0 51.7
    WING_HDG_FILTERED_MEAN          (time) float64 91.8 96.0 95.8 ... 53.1 52.5
    WING_HDG_FILTERED_STDDEV        (time) float64 3.2 3.9 3.8 ... 2.9 4.8 4.5
    WING_ROLL_FILTERED_MEAN         (time) float64 2.8 3.3 3.4 ... -0.3 0.3 0.0
    WING_ROLL_FILTERED_STDDEV       (time) float64 3.7 4.6 5.1 ... 3.2 2.8 3.1
    WING_ROLL_FILTERED_PEAK         (time) float64 nan nan nan ... nan nan nan
    WING_PITCH_FILTERED_MEAN        (time) float64 3.6 3.6 3.8 ... 1.5 1.6 1.0
    WING_PITCH_FILTERED_STDDEV      (time) float64 4.9 4.6 4.8 ... 6.9 9.6 6.9
    WING_PITCH_FILTERED_PEAK        (time) float64 14.1 13.1 14.6 ... 24.4 19.5
    WING_ANGLE                      (time) float64 -57.1 -52.6 ... 99.2 99.9
    UWND_MEAN                       (time) float64 -7.54 -7.74 ... -7.27 -7.47
    UWND_STDDEV                     (time) float64 0.76 0.62 0.48 ... 0.75 0.51
    VWND_MEAN                       (time) float64 -2.94 -2.88 ... -1.27 -1.53
    VWND_STDDEV                     (time) float64 0.73 0.58 0.51 ... 0.58 0.48
    WWND_MEAN                       (time) float64 -0.05 -0.08 ... -0.17 -0.12
    WWND_STDDEV                     (time) float64 0.39 0.38 0.33 ... 0.36 0.33
    GUST_WND_MEAN                   (time) float64 9.46 9.14 9.28 ... 8.34 8.36
    GUST_WND_STDDEV                 (time) float64 0.66 0.42 0.38 ... 0.57 0.36
    WIND_MEASUREMENT_HEIGHT_MEAN    (time) float64 4.954 4.946 ... 4.945 4.946
    WIND_MEASUREMENT_HEIGHT_STDDEV  (time) float64 0.052 0.07 ... 0.074 0.06
    TEMP_AIR_MEAN                   (time) float64 27.01 26.95 ... 26.15 26.12
    TEMP_AIR_STDDEV                 (time) float64 0.02 0.03 0.03 ... 0.06 0.04
    RH_MEAN                         (time) float64 79.03 78.27 ... 79.0 78.45
    RH_STDDEV                       (time) float64 0.48 0.48 0.63 ... 0.81 0.46
    BARO_PRES_MEAN                  (time) float64 1.016e+03 ... 1.014e+03
    BARO_PRES_STDDEV                (time) float64 0.08 0.09 0.09 ... 0.06 0.06
    PAR_AIR_MEAN                    (time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PAR_AIR_STDDEV                  (time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (time) float64 26.77 26.76 ... 26

# Read in SMAP from mapped podaac drive (z:)

In [79]:
#RSS
#get list of all filenames in directory
syr = str(ds_usv.time[0].dt.year.data).zfill(4)
adir = 'Z:/SalinityDensity/smap/L3/RSS/V4/8day_running/SCI/'+syr+'/*/*.nc'
files = [x for x in glob(adir)]
print('number of file:',len(files))

#don't read in days when no data from usv
files_filtered = []
for idy in range(int(ds_usv.time[0].dt.dayofyear),int(ds_usv.time[-1].dt.dayofyear+1)):
    sdy = '\\'+str(idy).zfill(3)+'\\'
    matches = [x for x in files if sdy in x]
    files_filtered.append(matches)
    
# read in data
ds = xr.open_mfdataset(files_filtered,combine='nested',concat_dim='time')
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds.close()  
ds_rss = ds

number of file: 218


ValueError: concat_dims has length 1 but the datasets passed are nested in a 2-dimensional structure

In [103]:
# read in data
print(files_filtered[0])
tt = str(files_filtered[0]).replace("\\","/").replace("//","/")
print(tt[0])
tt = 'Z:/SalinityDensity/smap/L3/RSS/V4/8day_running/SCI/2020/017/RSS_smap_SSS_L3_8day_running_2020_021_FNL_v04.0.nc'
print(tt)
ds2 = xr.open_dataset(tt)  #combine='nested',concat_dim='time')
#ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
#ds = ds.sortby(ds.lon)
#ds.close()  
#ds_rss = ds
ds2

['Z:/SalinityDensity/smap/L3/RSS/V4/8day_running/SCI/2020\\017\\RSS_smap_SSS_L3_8day_running_2020_021_FNL_v04.0.nc']
[
Z:/SalinityDensity/smap/L3/RSS/V4/8day_running/SCI/2020/017/RSS_smap_SSS_L3_8day_running_2020_021_FNL_v04.0.nc


<xarray.Dataset>
Dimensions:                         (obs: 66240, trajectory: 1)
Coordinates:
    latitude                        (trajectory, obs) float64 ...
    longitude                       (trajectory, obs) float64 ...
    time                            (trajectory, obs) datetime64[ns] ...
  * trajectory                      (trajectory) float32 1061.0
Dimensions without coordinates: obs
Data variables:
    SOG                             (trajectory, obs) float64 ...
    SOG_FILTERED_MEAN               (trajectory, obs) float64 ...
    SOG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    SOG_FILTERED_MAX                (trajectory, obs) float64 ...
    SOG_FILTERED_MIN                (trajectory, obs) float64 ...
    COG                             (trajectory, obs) float64 ...
    COG_FILTERED_MEAN               (trajectory, obs) float64 ...
    COG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    HDG                             (trajectory, obs) float64 ...
    HDG_FILTERED_MEAN               (trajectory, obs) float64 ...
    HDG_FILTERED_STDDEV             (trajectory, obs) float64 ...
    ROLL_FILTERED_MEAN              (trajectory, obs) float64 ...
    ROLL_FILTERED_STDDEV            (trajectory, obs) float64 ...
    ROLL_FILTERED_PEAK              (trajectory, obs) float64 ...
    PITCH_FILTERED_MEAN             (trajectory, obs) float64 ...
    PITCH_FILTERED_STDDEV           (trajectory, obs) float64 ...
    PITCH_FILTERED_PEAK             (trajectory, obs) float64 ...
    HDG_WING                        (trajectory, obs) float64 ...
    WING_HDG_FILTERED_MEAN          (trajectory, obs) float64 ...
    WING_HDG_FILTERED_STDDEV        (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_MEAN         (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_STDDEV       (trajectory, obs) float64 ...
    WING_ROLL_FILTERED_PEAK         (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_MEAN        (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_STDDEV      (trajectory, obs) float64 ...
    WING_PITCH_FILTERED_PEAK        (trajectory, obs) float64 ...
    WING_ANGLE                      (trajectory, obs) float64 ...
    UWND_MEAN                       (trajectory, obs) float64 ...
    UWND_STDDEV                     (trajectory, obs) float64 ...
    VWND_MEAN                       (trajectory, obs) float64 ...
    VWND_STDDEV                     (trajectory, obs) float64 ...
    WWND_MEAN                       (trajectory, obs) float64 ...
    WWND_STDDEV                     (trajectory, obs) float64 ...
    GUST_WND_MEAN                   (trajectory, obs) float64 ...
    GUST_WND_STDDEV                 (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN    (trajectory, obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_STDDEV  (trajectory, obs) float64 ...
    TEMP_AIR_MEAN                   (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV                 (trajectory, obs) float64 ...
    RH_MEAN                         (trajectory, obs) float64 ...
    RH_STDDEV                       (trajectory, obs) float64 ...
    BARO_PRES_MEAN                  (trajectory, obs) float64 ...
    BARO_PRES_STDDEV                (trajectory, obs) float64 ...
    PAR_AIR_MEAN                    (trajectory, obs) float64 ...
    PAR_AIR_STDDEV                  (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (trajectory, obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (trajectory, obs) float64 ...
    WAVE_DOMINANT_PERIOD            (trajectory, obs) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (trajectory, obs) float64 ...
    TEMP_SBE37_MEAN                 (trajectory, obs) float64 ...
    TEMP_SBE37_STDDEV               (trajectory, obs) float64 ...
    SAL_SBE37_MEAN                  (trajectory, obs) float64 ...
    SAL_SBE37_STDDEV                (trajectory, obs) float64 ...
    COND_SBE37_MEAN                 (trajectory, obs) float64 ...
    COND_SBE37_STDDEV

In [ ]:
#JPL
#get list of all filenames in directory
syr = str(ds_usv.time[0].dt.year.data).zfill(4)
adir = 'Z:/SalinityDensity/smap/L3/JPL/V4.3/8day_running/SCI/'+syr+'/*/4.3.nc'
files = [x for x in glob(adir)]
print('number of file:',len(files))

#don't read in days when no data from usv
files_filtered = []
for idy in range(int(ds_usv.time[0].dt.dayofyear),int(ds_usv.time[-1].dt.dayofyear+1)):
    sdy = '\\'+str(idy).zfill(3)+'\\'
    matches = [x for x in files if sdy in x]
    files_filtered.append(matches)
    
# read in data
ds = xr.open_mfdataset(files_filtered,combine='nested',concat_dim='time')
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds.close()  
ds_jpl = ds

# Collocate SMAP RSS 8day

In [ ]:
%%time
ds_usv['lat'] = ds_usv.lat.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
ds_usv['lon'] = ds_usv.lon.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
tem = ds_usv.lat.ffill(dim='time')
tem = ds_usv.lat.bfill(dim='time')
t1,t2=ds_usv.time.min().data-np.timedelta64(8,'D'),ds_usv.time.max().data+np.timedelta64(8,'D')
x1,x2=ds_usv.lon.min().data,ds_usv.lon.max().data
y1,y2=ds_usv.lat.min().data,ds_usv.lat.max().data
print(t1,t2)
ds_sat = ds.sel(time=slice(t1,t2),lat=slice(y1,y2),lon=slice(x1,x2)).load()   
ds_interp = ds_sat.interp(time=ds_usv.time,lat=ds_usv.lat,lon=ds_usv.lon,method='linear')#.interp(method='nearest')
#fout = dir_out+name+'_RSS8dy'+'.nc'
#ds_interp.to_netcdf(fout)

# Collocation SMAP JPL 8day

In [ ]:
#JPL
adir = 'F:/data/sat_data/smap/SSS/L3/JPL/V4.3/8day_running/**/**/*4.3.nc'
files = [x for x in glob(adir)]
print('number of file:',len(files))

ds = xr.open_mfdataset(files,combine='nested',concat_dim='time')
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds = ds.sortby(ds.lat)
ds.close()  
ds

In [ ]:
%%time
ds_usv['lat'] = ds_usv.lat.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
ds_usv['lon'] = ds_usv.lon.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
tem = ds_usv.lat.ffill(dim='time')
tem = ds_usv.lat.bfill(dim='time')
t1,t2=ds_usv.time.min().data-np.timedelta64(8,'D'),ds_usv.time.max().data+np.timedelta64(8,'D')
x1,x2=ds_usv.lon.min().data,ds_usv.lon.max().data
y1,y2=ds_usv.lat.min().data,ds_usv.lat.max().data
print(t1,t2)
ds_sat = ds.sel(time=slice(t1,t2),lat=slice(y1,y2),lon=slice(x1,x2)).load()   
ds_interp = ds_sat.interp(time=ds_usv.time,lat=ds_usv.lat,lon=ds_usv.lon,method='linear')#.interp(method='nearest')
#fout = dir_out+name+'_JPL8dy'+'.nc'
#ds_interp.to_netcdf(fout)